<a href="https://colab.research.google.com/github/Abdulkareem777/Abdulkareem/blob/main/Mobile_Net_V1_CIFAR_100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torchvision import models
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
num_epochs = 100
batch_size = 128
learning_rate = 0.001

cuda


In [2]:
transform = transforms.Compose([
    transforms.Resize(size=(224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
       (0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)
    )
])
train_dataset = torchvision.datasets.CIFAR100(
    root= './data', train = True,
    download =True, transform = transform)
test_dataset = torchvision.datasets.CIFAR100(
    root= './data', train = False,
    download =True, transform = transform)

100%|██████████| 169M/169M [00:13<00:00, 12.8MB/s]


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


In [3]:
train_loader = torch.utils.data.DataLoader(train_dataset
    , batch_size = batch_size
    , shuffle = True)
test_loader = torch.utils.data.DataLoader(test_dataset
    , batch_size = batch_size
    , shuffle = True)
n_total_step = len(train_loader)
print(n_total_step)

391


In [4]:
pip install torch-summary

In [5]:
from torchsummary import summary

In [6]:
class DepthwiseConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(DepthwiseConvBlock, self).__init__()
        self.depthwise_conv = nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=stride, padding=1, groups=in_channels, bias=False)
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.pointwise_conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.depthwise_conv(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.pointwise_conv(x)
        x = self.bn2(x)
        x = self.relu(x)
        return x

class MobileNetV1(nn.Module):
    def __init__(self, num_classes=100):
        super(MobileNetV1, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu = nn.ReLU(inplace=True)
        self.features = nn.Sequential(
            DepthwiseConvBlock(32, 64, stride=1),
            DepthwiseConvBlock(64, 128, stride=2),
            DepthwiseConvBlock(128, 128, stride=1),
            DepthwiseConvBlock(128, 256, stride=2),
            DepthwiseConvBlock(256, 256, stride=1),
            DepthwiseConvBlock(256, 512, stride=2),
            DepthwiseConvBlock(512, 512, stride=1),
            DepthwiseConvBlock(512, 512, stride=1),
            DepthwiseConvBlock(512, 512, stride=1),
            DepthwiseConvBlock(512, 512, stride=1),
            DepthwiseConvBlock(512, 512, stride=1),
            DepthwiseConvBlock(512, 1024, stride=2),
            DepthwiseConvBlock(1024, 1024, stride=1),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(1024, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [7]:
# model = models.vgg16(pretrained = True)
# layers = list(model.features.children())
# if using pretrained then only train conv5_1, conv5_2, and conv5_3
# for l in layers[:-5]:
#   for p in l.parameters():
#     p.requires_grad = False
# layers = list(model.features.children())
# if using pretrained then only train conv5_1, conv5_2, and conv5_3
# for l in layers[:-5]:
#   for p in l.parameters():
#     p.requires_grad = False
# input_lastLayer = model.classifier[6].in_features
# model.classifier[6] = nn.Linear(input_lastLayer,10)
model = MobileNetV1().to(device)
vgg = model.to(device)
print("******************************************************************************")
for name, param in vgg.named_parameters():
    print(name, param.requires_grad)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate, momentum=0.9,weight_decay=5e-4)

******************************************************************************
conv1.weight True
bn1.weight True
bn1.bias True
features.0.depthwise_conv.weight True
features.0.bn1.weight True
features.0.bn1.bias True
features.0.pointwise_conv.weight True
features.0.bn2.weight True
features.0.bn2.bias True
features.1.depthwise_conv.weight True
features.1.bn1.weight True
features.1.bn1.bias True
features.1.pointwise_conv.weight True
features.1.bn2.weight True
features.1.bn2.bias True
features.2.depthwise_conv.weight True
features.2.bn1.weight True
features.2.bn1.bias True
features.2.pointwise_conv.weight True
features.2.bn2.weight True
features.2.bn2.bias True
features.3.depthwise_conv.weight True
features.3.bn1.weight True
features.3.bn1.bias True
features.3.pointwise_conv.weight True
features.3.bn2.weight True
features.3.bn2.bias True
features.4.depthwise_conv.weight True
features.4.bn1.weight True
features.4.bn1.bias True
features.4.pointwise_conv.weight True
features.4.bn2.weight Tru

In [8]:
best_loss = np.inf
patience = 5
counter = 0
for epoch in range(num_epochs):
    for i, (imgs , labels) in enumerate(train_loader):
      imgs = imgs.to(device)
      labels = labels.to(device)
      labels_hat = model(imgs)
      n_corrects = (labels_hat.argmax(axis=1)==labels).sum().item()
      loss_value = criterion(labels_hat, labels)
      loss_value.backward()
      optimizer.step()
      optimizer.zero_grad()
      if (i+1) % 250 == 0:
        print(f'epoch {epoch+1}/{num_epochs}, step: {i+1}/{n_total_step}: loss = {loss_value:.5f}, acc = {100*(n_corrects/labels.size(0)):.2f}%')
        print()
    print('val')
    with torch.no_grad():
      number_corrects = 0
      number_samples = 0
      val_loss = 0
      for i, (test_images_set , test_labels_set) in enumerate(test_loader):
          test_images_set = test_images_set.to(device)
          test_labels_set = test_labels_set.to(device)

          y_predicted = model(test_images_set)
          val_loss += criterion(y_predicted, test_labels_set)
          labels_predicted = y_predicted.argmax(axis = 1)
          number_corrects += (labels_predicted==test_labels_set).sum().item()
          number_samples += test_labels_set.size(0)
      print(f'Overall accuracy {(number_corrects / number_samples)*100}%')
      # Update the best loss and the counter
      if val_loss < best_loss:
        best_loss = val_loss
        counter = 0
        # Save the best model weights
        torch.save(model, 'best_model.pth.tar')
      else:
        counter += 1

      # Check if the patience has been exceeded
    if counter >= patience:
      print('Early stopping!')
      break


epoch 1/100, step: 250/391: loss = 4.47857, acc = 5.47%

val
Overall accuracy 3.1399999999999997%
epoch 2/100, step: 250/391: loss = 4.30964, acc = 3.12%

val
Overall accuracy 5.08%
epoch 3/100, step: 250/391: loss = 4.08243, acc = 7.03%

val
Overall accuracy 7.46%
epoch 4/100, step: 250/391: loss = 3.98204, acc = 8.59%

val
Overall accuracy 8.58%
epoch 5/100, step: 250/391: loss = 3.90873, acc = 10.16%

val
Overall accuracy 10.33%
epoch 6/100, step: 250/391: loss = 3.93912, acc = 15.62%

val
Overall accuracy 12.15%
epoch 7/100, step: 250/391: loss = 3.67286, acc = 10.94%

val
Overall accuracy 13.19%
epoch 8/100, step: 250/391: loss = 3.61384, acc = 13.28%

val
Overall accuracy 15.49%
epoch 9/100, step: 250/391: loss = 3.35933, acc = 17.97%

val
Overall accuracy 16.85%
epoch 10/100, step: 250/391: loss = 3.41100, acc = 19.53%

val
Overall accuracy 18.77%
epoch 11/100, step: 250/391: loss = 3.30442, acc = 18.75%

val
Overall accuracy 19.67%
epoch 12/100, step: 250/391: loss = 2.91533, a

In [9]:
with torch.no_grad():
    number_corrects = 0
    number_samples = 0
    for i, (test_images_set , test_labels_set) in enumerate(test_loader):
        test_images_set = test_images_set.to(device)
        test_labels_set = test_labels_set.to(device)

        y_predicted = model(test_images_set)
        labels_predicted = y_predicted.argmax(axis = 1)
        number_corrects += (labels_predicted==test_labels_set).sum().item()
        number_samples += test_labels_set.size(0)
    print(f'Overall accuracy {(number_corrects / number_samples)*100}%')

Overall accuracy 39.290000000000006%


In [10]:
torch.save(model, 'model.pth.tar')